In [1]:
import pandas as pd
from pandas import DataFrame, Series
data = pd.read_csv('20220314_d3Neuron_pooled_DDA1_Peptide.csv')
data

,Annotated Sequence,Modifications,Contaminant,# Protein Groups,# Proteins,# PSMs,Master Protein Accessions,Positions in Master Proteins,Theo. MH+ [Da],m/z [Da] (by Search Engine): Sequest HT,RT [min] (by Search Engine): Sequest HT
0,[K].AAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSGPA...,NaN,False,1,1,1,O60341,O60341 [7-58],4402.11417,1101.29041,90.9539
1,[K].LQVDALIDTIKK.[S],NaN,False,1,1,2,Q92614,Q92614 [1027-1038],1356.80969,452.94354,84.1757
2,[R].LQVAGEITTGPR.[V],NaN,False,2,2,1,O75746; Q9UJS0,O75746 [454-465]; Q9UJS0 [456-467],1241.68483,621.34930,57.7644
3,[R].IQTQPGYANTLR.[D],NaN,False,1,1,1,Q00325,Q00325 [190-201],1361.71719,681.36566,47.6564
4,[R].IQTLPNQNQSQTQPLLK.[T],NaN,False,1,1,1,Q8IX12,Q8IX12 [205-221],1951.06071,976.03992,65.0931
...,...,...,...,...,...,...,...,...,...,...,...
28181,[R].GQLLAQLGDSARHRPR.[R],NaN,False,1,1,1,O60391,O60391 [1005-1020],1774.97832,887.99280,131.5401
28182,[R].ILTTSSAHPGLEFSK.[T],NaN,False,1,1,1,Q7L8L6,Q7L8L6 [68-82],1587.83770,529.95868,62.5013
28183,[R].QEDKDDLDVTELTNEDLLDQLVK.[Y],NaN,False,2,2,1,P42167; P42166,P42167 [104-126]; P42166 [104-126],2688.30979,1344.66516,122.5677
28184,[R].EVPVISASTQIMLK.[G],1xOxidation [M12],False,1,1,1,Q9BX63,Q9BX63 [515-528],1531.84001,766.42438,108.0715


In [2]:
import numpy as np
from sklearn.neighbors import LocalOutlierFactor

In [4]:
#The column number may change based on your own data file. 
x = data.iloc[:,9] #m/z
y = data.iloc[:,10] #RT
z = data.iloc[:,2] #Contaminant
h = data.iloc[:,6] #protein
m = data.iloc[:,0] #peptide sequence

In [5]:
x

0        1101.29041
1         452.94354
2         621.34930
3         681.36566
4         976.03992
            ...    
28181     887.99280
28182     529.95868
28183    1344.66516
28184     766.42438
28185     495.27377
Name: m/z [Da] (by Search Engine): Sequest HT, Length: 28186, dtype: float64

In [6]:
type(x)

pandas.core.series.Series

In [7]:
vector_list=[]
for i in range(0,28186): #the range here needs to be changed by the length of your list
    vector=[x[i],y[i]]
    vector_list.append(vector)

In [8]:
# Here is to adjust n_neighbors parameter
clf = LocalOutlierFactor(n_neighbors=200)
clf.fit_predict(vector_list)
result = clf.fit_predict(vector_list)

In [9]:
result

array([1, 1, 1, ..., 1, 1, 1])

In [10]:
clf.negative_outlier_factor_

array([-1.02983631, -1.04289254, -0.99915074, ..., -1.28037462,
       -0.99941862, -1.00425224])

In [11]:
#Briefly check how mnay outliers in your data.
count = 0
for i in result:
    if i == -1:
        count +=1
count

175

In [15]:
import csv
last_row=len(data)
with open('072225_d3 neuron_n=200_Example for paper.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['m/z','RT','fit_predict','negative outlier factor','Uniprot ID','Contamination','Peptide Sequence'])
    for a in range(0,last_row):
        c1=x[a]
        c2=y[a]
        c3=result[a]
        c4=clf.negative_outlier_factor_[a]
        c5=h[a]
        c6=z[a]
        c7=m[a]
        writer.writerow([c1,c2,c3,c4,c5,c6,c7])